In [ ]:
""" Script to test the flask API
"""
from ast import literal_eval
import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Base variables and methods
url_base_path = "http://zeus.itleiria.pt:5000/" 

def get_rgb_from_yuv(raw, shape):
    """Convert YUV420 to RGB array!

    :param raw: bytearray received by API
    :type raw: bytearray
    """
    yuv = np.frombuffer(raw, dtype=np.uint8).reshape(shape)
    return cv2.cvtColor(yuv, cv2.COLOR_YUV420P2BGR)  # YV12)

def display_frame_raw(r_content,shape):
    # convert yuv to rgb
    if len(r_content) > 0:
        rgb = get_rgb_from_yuv(r_content, shape)
        # display image
        plt.imshow(rgb)
        plt.show()
    else:
        print("Received an empty bytearray!")


In [ ]:
## 1. Print List of Streams

# 1.1 Get stream and print results
print("1.1 Get stream and print results")
r = requests.get(url_base_path + "get_stream_list")
for i, s in enumerate(literal_eval(r.content.decode())):
    print({"name": s[0], "width": s[1], "height": s[2], "bytes_per_pixel": s[3], "number_of_layers": s[4]})

print()
## 1.2. Get projections (get_projections)
print("1.2. Get projections (get_projections)")
r = requests.get(url_base_path + "get_projection_list")
for i, s in enumerate(literal_eval(r.content.decode())):
    print({"initial": s[0], "name": s[1]})

print()
## 1.3. Get projections faces (get_projections)
print("1.2. Get number of projection faces (get_number_faces)")
r = requests.get(url_base_path + "get_number_faces?projection=CMP")
for i, s in enumerate(literal_eval(r.content.decode())):
    print({"number of faces": s[0] })



In [ ]:
# 2 Select stream
print("2.1 Select stream")

stream_idx = 4
r = requests.get(url_base_path + "select_stream?idx="+str(stream_idx))  # post?
print(r.status_code)

print("2.1 Get layer info of current stream")
r = requests.get(url_base_path + "get_layer_info?idx="+str(stream_idx))
for i, s in enumerate(literal_eval(r.content.decode())):
    print({"layer": s[0], "description": s[1]})


In [ ]:
def get_frame(proj, layer=-1):
    """ Auxiliar method to: get_frame_info proj, get_frame_raw proj, display image."""
    # get frame info for ERP
    r_info = requests.get(url_base_path + "get_frame_info?projection="+proj+"&layer="+str(layer))
    frame_info = literal_eval(r_info.content.decode())
    print("Frame info:", frame_info)

    width = frame_info["width"]
    height = frame_info["height"]
    shape = (int(height * 1.5), width)

    # Get frame raw
    r_content = requests.get(url_base_path + "get_frame_raw?projection="+proj+"&layer="+str(layer), stream=True).content
    display_frame_raw(r_content,shape)

def get_two_frames(proj, layer=-1):
    """ Auxiliar method to: get_frame_info proj, get_frame_raw proj, display image."""
    # get frame info for ERP
    r_info = requests.get(url_base_path + "get_frame_info?projection="+proj+"&layer="+str(layer))
    frame_info = literal_eval(r_info.content.decode())
    print("Frame info:", frame_info)

    width = frame_info["width"]
    height = frame_info["height"]
    shape = (int(height * 1.5), width)

    # Get frame raw
    r_content = requests.get(url_base_path + "get_frame_raw?projection="+proj+"&layer="+str(layer), stream=True).content
    display_frame_raw(r_content,shape)

    r_content = requests.get(url_base_path + "get_frame_raw?projection="+proj+"&layer="+str(layer), stream=True).content
    display_frame_raw(r_content,shape)

print("4.1 Get projections (get_projections) - ERP")
get_two_frames("ERP")
get_two_frames("ERP")


In [ ]:
print("4.2 Get projections (get_projections) - CMP")
get_frame("CMP")
get_two_frames("CMP")

print("4.3 Get projections (get_projections) - CISP")
get_two_frames("CISP")
get_two_frames("CISP")

In [ ]:
## 5. Test Viewport
def get_viewport(x, y, w, h):
    """ Auxiliar method to: get_frame_info proj, get_frame_raw proj, display image."""

    viewport_api_args = "coord=pixel"+"&x="+str(x)+"&y="+str(y)+"&width="+str(w)+"&height="+str(h)

    r_info = requests.get(url_base_path + "get_viewport_info?" + viewport_api_args)
    frame_info = literal_eval(r_info.content.decode())
    print("Frame info:", frame_info)

    width = frame_info["width"]
    height = frame_info["height"]
    shape = (int(height * 1.5), width)

    # Get viewport raw
    r_content = requests.get(url_base_path + "get_viewport_raw?" + viewport_api_args, stream=True).content
    display_frame_raw(r_content,shape)

print("5.1 Get viewport")
get_viewport( 2347, 792, 469, 429 )

In [ ]:
print("6.1 Get projections in different layer (get_projections)")
get_frame("ERP", 0 )
get_frame("ERP", 0 )
get_frame("ERP", 1 )
get_frame("ERP", 2 )
get_frame("ERP", 0 )

In [ ]:
# 7.1 Select stream
print("7.1 Select new stream stream")
stream_idx = 2
r = requests.get(url_base_path + "select_stream?idx="+str(stream_idx))  # post?
print(r.status_code)

print("7.2 Get projection ERP")
get_frame("ERP")
print("7.3 Get projection ERP - Layer 1")
get_frame("ERP", layer = 1)
print("7.4 Get projection ERP - Layer 0")
get_frame("ERP", layer = 2)

In [ ]:
## 8. Test Cube Face
def get_face(proj, layer, f):
    """ Auxiliar method to: get_projection_face_info proj, get_projection_face_raw proj, display image."""

     # get face info for CMP
    r_info = requests.get(url_base_path + "get_projection_face_info?projection="+proj+"&layer="+str(layer)+"&face="+str(f))
    frame_info = literal_eval(r_info.content.decode())
    print("Frame info:", frame_info)

    width = frame_info["width"]
    height = frame_info["height"]
    shape = (int(height * 1.5), width)

    # Get face raw
    r_content = requests.get(url_base_path + "get_projection_face_raw?projection="+proj+"&layer="+str(layer)+"&face="+str(f), stream=True).content
    display_frame_raw(r_content,shape)


## 1.3. Get projections faces (get_projections)
print("1.2. Get number of projection faces (get_number_faces)")
r = requests.get(url_base_path + "get_number_faces?projection=CMP")
for i, s in enumerate(literal_eval(r.content.decode())):
    print({"number of faces": s[0] })

stream_idx = 3
r = requests.get(url_base_path + "select_stream?idx="+str(stream_idx))  # post?
print(r.status_code)

get_face("CMP", layer, 3)
get_face("CMP", layer, 4)

layer = -1
for face in range( int(s[0]) ):
    get_face("CMP", layer, face)